# PyCUDA installation

In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 15.1 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.2-cp312-cp312-linux_x86_64.whl size=659050 sha256=ec907766d68f3114cad7285a72b11acb6ef8fad78b568796349b69400d8802cc
  Stored in directory: /root/.cache/pip/wheels/d5/36/f3/ac5f09d768cad3fa15d5a3449bdfe65c3de58e69d036c73228
Successfully built pycuda




---



# Version #3: using ```gpuArrays```


The following initial code portion is the same as for Version #1 and #2.

In [2]:
import numpy as np

# --- PyCUDA initialization
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit

########
# MAIN #
########

start = cuda.Event()
end   = cuda.Event()

N = 100000

h_a = np.random.randn(N).astype(np.float32)
h_b = np.random.randn(N).astype(np.float32)

h_c = np.empty_like(h_a)

This version uses the ```gpuarray``` class. In this way, it is possible to allocate and move host memory space to device by ```gpuarray.to_gpu()```, perform the sum of the two arrays by simply using ```d_c = (d_a + d_b)``` and finally to move the result to host by the ```.get()``` method. There is no explicit declaration of ```d_c``` which automatically occurs during the execution of the ```d_c = (d_a + d_b)``` instruction. The gpuarray class internally compiles an elementwise kernel for arithmetic operations, allowing expressions like ```d_c = (d_a + d_b)``` to execute directly on the GPU without explicit kernel code.


In [3]:
d_a = gpuarray.to_gpu(h_a)
d_b = gpuarray.to_gpu(h_b)

# --- Warmup execution
d_c = (d_a + d_b)

cuda.Context.synchronize()
start.record()
d_c = d_a + d_b
end.record()
end.synchronize()
secs = start.time_till(end) * 1e-3
print("Processing time = %fs" % (secs))

h_c = d_c.get()

Processing time = 0.000450s


This last part is the same as for the previous versions.

In [4]:
if np.array_equal(h_c, h_a + h_b):
  print("Test passed!")
else :
  print("Error!")

cuda.Context.synchronize()

Test passed!
